In [16]:
import os
import google.generativeai as genai
import pymysql
from dotenv import load_dotenv

def db_connection():
      
    load_dotenv()

    DATABASE_IP = os.getenv("DATABASE_IP")
    DATABASE_USER = os.getenv("DATABASE_USER")
    DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
    DATABASE = os.getenv("DATABASE")

    db_connection = pymysql.connect(
        host=DATABASE_IP,       
        user=DATABASE_USER,            
        password=DATABASE_PASSWORD,      
        database=DATABASE 
    )

    return db_connection

def model_start():

    connection = db_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT API_KEY, Model FROM STAREX.AI_SetupMain")
    result = cursor.fetchall()

    API_KEY = result[0][0]
    model = result[0][1]

    api = os.environ["GOOGLE_API_KEY"] = API_KEY

    genai.configure(api_key=api)

    generation_config = {
      "temperature": 2,
      "top_p": 0.95,
      "top_k": 64,
      "max_output_tokens": 8192,
      "response_mime_type": "text/plain"
    }

    model = genai.GenerativeModel(model,
                                 generation_config=generation_config,)
    
    cursor.close()
    connection.close()
    
    return model

def system_prompt():
    
    load_dotenv()
    
    USER_NAME = os.getenv("USER_NAME")
    AI_NAME = os.getenv("AI_NAME")

    prompt = 'Senin adın ' + AI_NAME + ' ve seninle konuşan kişinin ismi' + USER_NAME

    return prompt

def get_chat_history():
    
    connection = db_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT Role, Description, CreateDate FROM STAREX.AI_PromptMain")
    result = cursor.fetchall()
    
    cursor.close()
    connection.close()

    return result

model = model_start()
connection = db_connection()
cursor = connection.cursor()
prompt = system_prompt()
    
chat_session = model.start_chat( history = [{'parts': [{'text':prompt}], 'role': 'assistant'}])

prompt =  input("")

if prompt != "exit":
    
    cursor.callproc("SP_STAREX_AIChat", ('user', prompt))
    connection.commit()

    response = chat_session.send_message(prompt, stream = True)
    response.resolve()
    
    cursor.callproc("SP_STAREX_AIChat", ('assistant', response.text))
    connection.commit()  
    
    print(response.text)
   
    while response.text:      
        prompt =  input("")
            
        if prompt == "exit":
                
            break
                
        cursor.callproc("SP_STAREX_AIChat", ('user', prompt))
        connection.commit()

        response = chat_session.send_message(prompt, stream = True)
        response.resolve()

        cursor.callproc("SP_STAREX_AIChat", ('assistant', response.text))
        connection.commit()  
    
        print(response.text)
    
cursor.close()
connection.close()

merhaba
Merhaba!  Nasıl yardımcı olabilirim?

exitr
Anladım.  Görüşmek üzere!

exit
